In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
from matplotlib import pyplot as plt
import yfinance as yf
import os
import plotly
import plotly.express as px
plotly.io.renderers.default = 'notebook'
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [41]:
def get_historical_data(tickers, time_period):
    #input: list of stock tickers
    #output: dataframe with all-time yahoo finance info

    tickers = [i.upper() for i in tickers]

    # try:
    df = yf.download(tickers, period = time_period)
    # except:
    #     print(f'{stock} data was not found. Fix symbol or try again.')

    for stock in tickers:
        df[('Delta', stock)] = df[('Close', stock)].pct_change()
        #simple moving averages
        df[('SMA10', stock)] = df[('Close', stock)].rolling(window=10).mean()
        df[('SMA50', stock)] = df[('Close', stock)].rolling(window=50).mean()
        df[('SMA200', stock)] = df[('Close', stock)].rolling(window=200).mean()
        #exponential moving averages
        df[('EMA10', stock)] = df[('Close', stock)].ewm(span=10).mean()
        df[('EMA50', stock)] = df[('Close', stock)].ewm(span=50).mean()
        df[('EMA200', stock)] = df[('Close', stock)].ewm(span=200).mean()
        #Bollinger bands
        df[('UpperBB', stock)] = df[('Close', stock)].rolling(window=20).mean() + (df[('Close', stock)].rolling(20).std() * 2)
        df[('LowerBB', stock)] = df[('Close', stock)].rolling(window=20).mean() - (df[('Close', stock)].rolling(20).std() * 2)
        
    return df

In [42]:
def create_returns(dataframe):
    chosen_stocks = dataframe.columns.get_level_values(1).unique()
    returns_list = [dataframe['Delta'][stock].rename(stock) for stock in chosen_stocks]
    returns_df = pd.concat(returns_list, axis=1)
    return returns_df

In [14]:
# def generate_corr_plot(ticker_list):
#     mask = np.triu(np.ones_like(dataframe.corr(), dtype=bool), k=1)
#     plt.figure(figsize=(8,6))
#     fig = sns.heatmap(dataframe.corr(), mask=mask, vmin=-1, cmap='coolwarm', annot=True)
#     plt.title('Historical Returns Correlation Matrix')
#     return fig

In [48]:
def generate_corr_plot(ticker_list, time_period_corr):
    data = yf.download(ticker_list, period=time_period_corr)['Close']
    data.dropna(axis=1, how='all', inplace=True)
    returns = data.pct_change().dropna()
    corr_matrix = returns.corr()

    fig = px.imshow(corr_matrix, text_auto=True,
                    color_continuous_scale='RdBu_r', zmin=-1, zmax=1,
                    title=f'Correlation: Historical Return ({time_period_corr})')
    fig.update_layout(height=600,width=600,paper_bgcolor='rgb(184, 201, 223)', coloraxis_colorbar = dict(x=1.1),
                      margin = dict(l=100))
    # corr_plot = plotly.io.to_html(fig, full_html=False)
    # return corr_plot.show()
    fig.show()

In [44]:
ticker_list = ['spy','aapl','msft']
time_period = '6mo'

# Historical Price Charts

In [140]:
df = get_historical_data(tickers, time_period)
df

[*********************100%***********************]  4 of 4 completed


Price            Close                                           High  \
Ticker            NVDA       RGTI         SYK        TWLO        NVDA   
Date                                                                    
2020-07-27   10.387262        NaN  187.298676  252.860001   10.407445   
2020-07-28   10.181937        NaN  184.394394  249.789993   10.347890   
2020-07-29   10.431117        NaN  191.161560  264.619995   10.481700   
2020-07-30   10.579131        NaN  190.013153  267.450012   10.588101   
2020-07-31   10.579879        NaN  183.464233  277.420013   10.731131   
2020-08-03   10.974075        NaN  181.850723  286.329987   11.062285   
2020-08-04   11.190865        NaN  179.667770  283.760010   11.190865   
2020-08-05   11.249669        NaN  178.395950  278.109985   11.334390   
2020-08-06   11.298259        NaN  178.993881  260.489990   11.320685   
2020-08-07   11.162706        NaN  181.793762  249.000000   11.466954   
2020-08-10   11.128320        NaN  178.120728  250.050003   11.370770   
2020-08-11   10.814355        NaN  179.307068  244.490005   11.100163   
2020-08-12   11.402667        NaN  180.502975  240.929993   11.429329   
2020-08-13   11.405406        NaN  181.243286  243.990005   11.683241   
2020-08-14   11.526007        NaN  178.785065  240.059998   11.666294   
2020-08-17   12.296469        NaN  179.411514  247.800003   12.368980   
2020-08-18   12.220469        NaN  179.724762  251.940002   12.454947   
2020-08-19   12.098620        NaN  176.630585  251.000000   12.274540   
2020-08-20   12.101114        NaN  178.301041  257.649994   12.334346   
2020-08-21   12.641829        NaN  179.174210  251.259995   12.766668   
2020-08-24   12.678459        NaN  179.098297  245.470001   12.870078   
2020-08-25   12.708114        NaN  179.924057  249.509995   12.719326   
2020-08-26   12.731038        NaN  179.392532  265.019989   12.826224   
2020-08-27   12.586764        NaN  186.216690  261.519989   12.807786   
2020-08-28   13.104557        NaN  188.124420  264.450012   13.104806   
2020-08-31   13.330563        NaN  188.076965  269.760010   13.530403   
2020-09-01   13.779717        NaN  187.953568  273.239990   13.951951   
2020-09-02   14.303647        NaN  191.360870  264.200012   14.682761   
2020-09-03   12.976622        NaN  188.057938  248.389999   13.843025   
2020-09-04   12.584795        NaN  187.564407  233.500000   13.135644   
2020-09-08   11.877415        NaN  189.899246  226.160004   12.512512   
2020-09-09   12.677020        NaN  194.958038  233.110001   12.770988   
2020-09-10   12.274975        NaN  191.512756  231.169998   13.026224   
2020-09-11   12.128162        NaN  191.740524  224.410004   12.614705   
2020-09-14   12.833799        NaN  195.052963  225.910004   13.275227   
2020-09-15   12.952196        NaN  199.789001  226.330002   13.259027   
2020-09-16   12.477118        NaN  204.828812  228.740005   13.053640   
2020-09-17   12.426270        NaN  200.804626  221.869995   12.552144   
2020-09-18   12.152839        NaN  195.878677  225.399994   12.594764   
2020-09-21   12.479859        NaN  192.585236  238.300003   12.485094   
2020-09-22   12.599998        NaN  193.021835  237.000000   12.610218   
2020-09-23   12.087534        NaN  192.101151  232.000000   12.688982   
2020-09-24   12.311114        NaN  188.836227  238.399994   12.531953   
2020-09-25   12.835295        NaN  194.578369  245.020004   12.845016   
2020-09-28   12.996064        NaN  196.704407  244.520004   13.012764   
2020-09-29   13.186243        NaN  194.862778  248.800003   13.402595   
2020-09-30   13.490081        NaN  198.317596  247.089996   13.581806   
2020-10-01   13.573831        NaN  198.603149  256.959991   13.721637   
2020-10-02   13.023232        NaN  198.251007  290.359985   13.479116   
2020-10-05   13.601748        NaN  203.733109  283.000000   13.603492   
2020-10-06   13.695468        NaN  200.430511  294.510010   14.099757   
2020-10-07   13.922286        NaN  206.911987  298.179993 

In [143]:
df.columns

MultiIndex([(  'Close', 'NVDA'),
            (  'Close', 'RGTI'),
            (  'Close',  'SYK'),
            (  'Close', 'TWLO'),
            (   'High', 'NVDA'),
            (   'High', 'RGTI'),
            (   'High',  'SYK'),
            (   'High', 'TWLO'),
            (    'Low', 'NVDA'),
            (    'Low', 'RGTI'),
            (    'Low',  'SYK'),
            (    'Low', 'TWLO'),
            (   'Open', 'NVDA'),
            (   'Open', 'RGTI'),
            (   'Open',  'SYK'),
            (   'Open', 'TWLO'),
            ( 'Volume', 'NVDA'),
            ( 'Volume', 'RGTI'),
            ( 'Volume',  'SYK'),
            ( 'Volume', 'TWLO'),
            (  'Delta', 'NVDA'),
            (  'SMA10', 'NVDA'),
            (  'SMA50', 'NVDA'),
            ( 'SMA200', 'NVDA'),
            (  'EMA10', 'NVDA'),
            (  'EMA50', 'NVDA'),
            ( 'EMA200', 'NVDA'),
            ('UpperBB', 'NVDA'),
            ('LowerBB', 'NVDA'),
            (  'Delta',  'SYK'),
          

In [163]:
tickers = ['nvda','syk','rgti', 'twlo']
time_period = '5y'
df = get_historical_data(tickers, time_period)

[*********************100%***********************]  4 of 4 completed


In [177]:
#consider using plotly graph objects here instead of plotly express
def get_time_series(df, ticker):
    df_flat = df.copy()
    df_flat.columns = ['-'.join(col).strip() for col in df.columns.values]
    fig = px.line(df_flat, x=df_flat.index, 
                  y=[f'Close-{ticker}', f'SMA10-{ticker}', f'SMA50-{ticker}', f'SMA200-{ticker}'],
                  title=f'${ticker} Historical Pricing', 
                  labels={'value':'Price/Share ($)', 'variabble':'Metric', f'Close-{ticker}':'Closing Price'})
    
    fig.update_layout(paper_bgcolor='rgb(184, 201, 223)')
    
    legend_names = ['Close', 'SMA10', 'SMA50', 'SMA200']
    for trace, name in zip(fig.data, legend_names):
        trace.name = name
        trace.legendgroup = name
        trace.hovertemplate = trace.hovertemplate.replace(trace.name, name)
    
    fig.show()

In [50]:
def get_time_series(df, ticker, time_period):
    df_flat = df.copy()
    df_flat.columns = ['-'.join(col).strip() for col in df.columns.values]
    fig = px.line(df_flat, x=df_flat.index, 
                  y=[f'Close-{ticker}', f'SMA10-{ticker}', f'SMA50-{ticker}', f'SMA200-{ticker}',
                    f'EMA10-{ticker}', f'EMA50-{ticker}', f'EMA200-{ticker}',
                    f'UpperBB-{ticker}', f'LowerBB-{ticker}'],
                  title=f'${ticker} Historical Pricing - {time_period}', 
                  labels={'value':'Price/Share ($)', 'variable':f'${ticker} Metrics'})
    
    fig.update_layout(paper_bgcolor='rgb(184, 201, 223)')
    
    legend_names = ['Close', 'SMA10', 'SMA50', 'SMA200', 
                    'EMA10', 'EMA50', 'EMA200', 'LowerBB', 'UpperBB']
    for trace, name in zip(fig.data, legend_names):
        trace.name = name
        trace.legendgroup = name
        trace.hovertemplate = trace.hovertemplate.replace(trace.name, name)

    for i, trace in enumerate(fig.data):
        trace.visible = True if i < 3 else "legendonly"

    # time_series_plot = plotly.io.to_html(fig, full_html=False)
    # return time_series_plot
    fig.show()


In [56]:
df = get_historical_data(['SPY'],'6mo')

get_time_series(df, 'SPY', time_period)

[*********************100%***********************]  1 of 1 completed


# Additional Info Table

In [72]:
def get_metrics(ticker_list):
    metrics_info = ['currentPrice','beta','marketCap']
    stock_names = []
    values_list = []

    for i in ticker_list:
        stock_names.append(yf.Ticker(i).info.get('shortName'))
        metric_list = [yf.Ticker(i).info.get(j) for j in metrics_info]
        try:
            earnings_element = yf.Ticker(i).calendar.get('Earnings Date')
            earnings_date = str(earnings_element[0])
        except Exception as e:
            earnings_date = pd.NA
        metric_list.append(earnings_date)
        values_list.append(metric_list)
        
        final_dict = dict(zip(stock_names, values_list))

    return final_dict

In [82]:
a = get_metrics(['^GSPC'])
a

{'S&P 500': [None, None, None, <NA>]}

In [88]:
def get_historical_data(tickers, time_period):
    ''' input: list of stock tickers and time period
        output: dataframe with all-time yahoo finance info


    '''

    tickers = [i.upper() for i in tickers]

    # try:
    df = yf.download(tickers, period=time_period)
    # except:
    #     print(f'{stock} data was not found. Fix symbol or try again.')

    for stock in tickers:
        #daily change
        df[('Delta', stock)] = df[('Close', stock)].pct_change()
        #simple moving average (10,50,200 day)
        df[('SMA10', stock)] = df[('Close', stock)].rolling(window=10).mean()
        df[('SMA50', stock)] = df[('Close', stock)].rolling(window=50).mean()
        df[('SMA200', stock)] = df[('Close', stock)].rolling(window=200).mean()
        #exponential moving average (more weight to recent prices)
        df[('EMA10', stock)] = df[('Close', stock)].ewm(span=10).mean()
        df[('EMA50', stock)] = df[('Close', stock)].ewm(span=50).mean()
        df[('EMA200', stock)] = df[('Close', stock)].ewm(span=200).mean()
        #Bollinger bands: 2 std devs above&below 20 day SMA
        df[('UpperBB', stock)] = df[('Close', stock)].rolling(window=20).mean() + (df[('Close', stock)].rolling(20).std() * 2)
        df[('LowerBB', stock)] = df[('Close', stock)].rolling(window=20).mean() - (df[('Close', stock)].rolling(20).std() * 2)
        
    return df

def create_returns(dataframe):
    """
        Takes the large dataframe and grabs the Delta columns to create a new dataframe solely 
        including delta (daily returns)
    """

    chosen_stocks = dataframe.columns.get_level_values(1).unique()
    returns_list = [dataframe['Delta'][stock].rename(stock) for stock in chosen_stocks]
    returns_df = pd.concat(returns_list, axis=1)
    return returns_df

def generate_corr_plot(ticker_list, time_period_corr):
    data = yf.download(ticker_list, period=time_period_corr)['Close']
    data.dropna(axis=1, how='all', inplace=True)
    returns = data.pct_change().dropna()
    corr_matrix = returns.corr()

    fig = px.imshow(corr_matrix, text_auto=True,
                    color_continuous_scale='RdBu_r', zmin=-1, zmax=1,
                    title=f'Correlation: Historical Return ({time_period_corr})')
    fig.update_layout(height=600,width=600,paper_bgcolor='rgb(184, 201, 223)', coloraxis_colorbar = dict(x=1.1),
                      margin = dict(r=0))
    # corr_plot = plotly.io.to_html(fig, full_html=False)
    # return corr_plot
    return fig.show()

# def get_beta(ticker_list):
#     beta_values = [yf.Ticker(i).info.get('beta') for i in ticker_list]
#     beta_dict = dict(zip(ticker_list, beta_values))
#     return beta_dict

def get_metrics(ticker_list):
    # metrics_info = ['currentPrice','beta','marketCap']
    metrics_info = ['regularMarketPrice','beta','marketCap']
    stock_names = []
    values_list = []

    for i in ticker_list:
        stock_names.append(yf.Ticker(i).info.get('shortName'))
        metric_list = [yf.Ticker(i).info.get(j) for j in metrics_info]
        try:
            earnings_element = yf.Ticker(i).calendar.get('Earnings Date')
            earnings_date = str(earnings_element[0])
        except Exception as e:
            earnings_date = 'None'
        metric_list.append(earnings_date)
        values_list.append(metric_list)
        
        final_dict = dict(zip(stock_names, values_list))
    
    for i in final_dict.values():
        if i[2] is not None and i[2] > 1000000000000:
            i[2] = f'{round(i[2] / 1000000000000, 2)}T'
        elif i[2] is not None and i[2] > 1000000000:
            i[2] = f'{round(i[2] / 1000000000,2)}B'
        elif i[2] is not None and i[2] > 1000000:
            i[2] = f'{round(i[2] / 1000000,2)}M'
        else:
            pass

    return final_dict

def get_time_series(df, ticker, time_period):
    df_flat = df.copy()
    df_flat.columns = ['-'.join(col).strip() for col in df.columns.values]
    fig = px.line(df_flat, x=df_flat.index, 
                  y=[f'Close-{ticker}', f'SMA10-{ticker}', f'SMA50-{ticker}', f'SMA200-{ticker}',
                    f'EMA10-{ticker}', f'EMA50-{ticker}', f'EMA200-{ticker}',
                    f'UpperBB-{ticker}', f'LowerBB-{ticker}'],
                  title=f'${ticker} Historical Pricing - {time_period}', 
                  labels={'value':'Price/Share ($)', 'variable':f'${ticker} Metrics'})
    
    fig.update_layout(paper_bgcolor='rgb(184, 201, 223)')
    
    legend_names = ['Close', 'SMA10', 'SMA50', 'SMA200', 
                    'EMA10', 'EMA50', 'EMA200', 'LowerBB', 'UpperBB']
    for trace, name in zip(fig.data, legend_names):
        trace.name = name
        trace.legendgroup = name
        trace.hovertemplate = trace.hovertemplate.replace(trace.name, name)

    for i, trace in enumerate(fig.data):
        trace.visible = True if i < 3 else "legendonly"

    # time_series_plot = plotly.io.to_html(fig, full_html=False)
    # return time_series_plot
    return fig.show()

In [89]:
# corr = generate_corr_plot(['^GSPC','SPY','BTC'], '1y')
metrics = get_metrics(['^GSPC','SPY','BTC'])

In [90]:
metrics

{'S&P 500': [6388.64, None, None, 'None'],
 'SPDR S&P 500': [637.1, None, '584.72B', 'None'],
 'Grayscale Bitcoin Mini Trust (B': [51.77, None, None, 'None']}

In [94]:
yf.Ticker('aapl').info

{'address1': 'One Apple Park Way',
 'city': 'Cupertino',
 'state': 'CA',
 'zip': '95014',
 'country': 'United States',
 'phone': '(408) 996-1010',
 'website': 'https://www.apple.com',
 'industry': 'Consumer Electronics',
 'industryKey': 'consumer-electronics',
 'industryDisp': 'Consumer Electronics',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and p

In [4]:
yf.Ticker('aapl').history('2dj')

AAPL: Period '2dj' is invalid, must be of the format 1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y, ytd, max, etc.


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
